## CBoW modell tanítása

Ez a jegyzetfüzet az [AI for Beginners Curriculum](http://aka.ms/ai-beginners) része.

Ebben a példában megvizsgáljuk, hogyan taníthatunk CBoW nyelvi modellt, hogy létrehozzuk saját Word2Vec beágyazási terünket. Az AG News adatállományt fogjuk használni szövegforrásként.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Először töltsük be az adatállományunkat, és definiáljuk a tokenizálót és a szókincset. A `vocab_size` értékét 5000-re állítjuk, hogy kissé korlátozzuk a számításokat.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW Modell

A CBoW megtanulja megjósolni egy szót a $2N$ szomszédos szavak alapján. Például, amikor $N=1$, a következő párokat kapjuk az *I like to train networks* mondatból: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Itt az első szó a szomszédos szó, amelyet bemenetként használunk, a második szó pedig az, amelyet megjósolunk.

Ahhoz, hogy egy hálózatot építsünk a következő szó megjóslására, szomszédos szót kell bemenetként megadnunk, és szószámot kell kimenetként kapnunk. A CBoW hálózat architektúrája a következő:

* A bemeneti szót az embedding rétegen keresztül adjuk át. Ez az embedding réteg lesz a Word2Vec embeddingünk, ezért külön `embedder` változóként definiáljuk. Ebben a példában az embedding méretét 30-ra állítjuk, bár érdemes lehet magasabb dimenziókkal kísérletezni (a valódi Word2Vec 300-at használ).
* Az embedding vektort ezután egy lineáris rétegbe továbbítjuk, amely megjósolja a kimeneti szót. Ezért ennek a rétegnek `vocab_size` neuronja van.

A kimenethez, ha `CrossEntropyLoss`-t használunk veszteségfüggvényként, akkor csak szószámokat kell megadnunk elvárt eredményként, egyhot kódolás nélkül.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Tanítóadatok előkészítése

Most programozzuk le a fő függvényt, amely a szövegből CBoW szó-párokat fog előállítani. Ez a függvény lehetővé teszi, hogy megadjuk az ablakméretet, és egy párokból álló halmazt ad vissza - bemeneti és kimeneti szavakat. Figyeljünk arra, hogy ez a függvény használható szavakon, valamint vektorokon/tensorokon is - ami lehetővé teszi, hogy a szöveget kódoljuk, mielőtt átadnánk a `to_cbow` függvénynek.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Készítsük elő a tanító adathalmazt. Átnézzük az összes hírt, meghívjuk a `to_cbow` függvényt, hogy megkapjuk a szópárok listáját, és hozzáadjuk ezeket a párokat az `X` és `Y` halmazokhoz. Időmegtakarítás érdekében csak az első 10k híreket vesszük figyelembe - ezt a korlátozást könnyen eltávolíthatod, ha több időd van várni, és jobb beágyazásokat szeretnél kapni :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Azt az adatot is egy adathalmazzá alakítjuk, és létrehozzuk az adatbetöltőt:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Azt az adatot is egy adathalmazzá alakítjuk, és létrehozunk egy adatbetöltőt:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Most kezdjük el a tényleges tanítást. `SGD` optimalizálót fogunk használni, viszonylag magas tanulási rátával. Kipróbálhatod más optimalizálókat is, például az `Adam`-et. Kezdetben 10 epochon keresztül fogunk tanítani - és újra futtathatod ezt a cellát, ha még alacsonyabb veszteséget szeretnél.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec kipróbálása

A Word2Vec használatához vegyük ki a vektorokat, amelyek a szókincsünk összes szavához tartoznak:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Lássuk például, hogyan van a **Párizs** szó vektorrá kódolva:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Érdekes a Word2Vec használata szinonimák keresésére. Az alábbi függvény visszaadja az adott bemenethez legközelebb eső `n` szót. Ezek megtalálásához kiszámítjuk $|w_i - v|$ normáját, ahol $v$ a bemeneti szóhoz tartozó vektor, és $w_i$ a szótár $i$-edik szavának kódolása. Ezután rendezzük a tömböt, és az `argsort` segítségével visszaadjuk a megfelelő indexeket, majd a lista első `n` elemét vesszük, amelyek a szótárban legközelebb eső szavak pozícióit kódolják.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Összegzés

Ügyes technikák, mint például a CBoW használatával, képezhetünk Word2Vec modellt. Kipróbálhatod a skip-gram modellt is, amelyet arra tanítanak, hogy a központi szó alapján megjósolja a szomszédos szavakat, és megnézheted, milyen jól teljesít.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget az ebből a fordításból eredő félreértésekért vagy téves értelmezésekért.
